## Register_raw_brains (python 3.6+)
The purpose of this notebook is to align multiple brains in their raw space by shifting them so that the center of mass of an atlas region is aligned. The procedure for this is as follows:
1. Each raw space brain gets a raw space atlas by back-transforming the atlas using the registration transformation parameters. 
2. One brain is chosen as the reference brain
3. For each brain (including reference brain), calculate the center of mass of a region on which you want to align the brains
4. Calculate the offset (dx,dy,dz) between the center of mass of this region between each brain and the reference brain.
5. Apply the offset to each of the brains to align them to the reference brain. 

We will use the 201908_cfos dataset as a test of this concept. Step 1 has already been run for this. Step 3 proof of concept is shown in [Center_of_mass_atlas.ipynb](Center_of_mass_atlas.ipynb). Step 5 proof of concept is shown in [shift_array.ipynb](shift_array.ipynb). 

In [2]:
import cloudvolume
import neuroglancer
import numpy as np
import os,glob
import time
import tifffile
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
raw_dir = '/jukebox/LightSheetTransfer/kelly/201908_cfos' # contains subdirectories with the raw data for each brain
atlas_dir = '/jukebox/scratch/kellyms' # contains subdirectories with the raw-space annotation atlases for each brain

## Aligning 2 brains 
Let's start simple and align two "observ" brains based on a given brain region

In [4]:
def find_observ_dirs(channel='647'):
    """ Find subdirectories of the root directory that contain
    "observ" and the channel name in them
    """
    assert type(channel) == str
    observ_dirs = glob.glob(raw_dir + '/*observ*%s*' % channel)
    return observ_dirs

In [5]:
find_observ_dirs()

['/jukebox/LightSheetTransfer/kelly/201908_cfos/190820_m61468_observ_20190628_1d3x_647_008na_1hfds_z5um_250msec_14-50-40',
 '/jukebox/LightSheetTransfer/kelly/201908_cfos/190821_f61465_observ_20190626_1d3x_647_008na_1hfds_z5um_250msec_12-20-20',
 '/jukebox/LightSheetTransfer/kelly/201908_cfos/190822_f62159_observ_20190625_1d3x_647_008na_1hfds_z5um_250msec_13-13-43',
 '/jukebox/LightSheetTransfer/kelly/201908_cfos/190821_f62196_observ_20190626_1d3x_647_008na_1hfds_z5um_250msec_10-50-16',
 '/jukebox/LightSheetTransfer/kelly/201908_cfos/190822_m61467_observ_20190702_1d3x_647_008na_1hfds_z5um_250msec_18-56-46',
 '/jukebox/LightSheetTransfer/kelly/201908_cfos/190822_m62181_observ_20190702_1d3x_647_008na_1hfds_z5um_250msec_16-02-35',
 '/jukebox/LightSheetTransfer/kelly/201908_cfos/190822_f61494_observ_20190625_1d3x_647_008na_1hfds_z5um_250msec_17-24-19']

In [6]:
# Pick first two as an example
all_observ_dirs = find_observ_dirs()
test_observ_dirs = all_observ_dirs[0:2]

In [7]:
# The string we need to find the raw space atlas is the "m61468_observ" part, so let's strip that out
test_animal_ids = ['_'.join(x.split('/')[-1].split('_')[1:3]) for x in test_observ_dirs]
test_animal_ids

['m61468_observ', 'f61465_observ']

In [20]:
# grab raw atlas files and put them in a list of lists
test_raw_atlas_files = []
for animal_id in test_animal_ids:
    subdirs = glob.glob(atlas_dir + f'/{animal_id}*')
    assert len(subdirs) == 1
    subdir = subdirs[0]
    raw_atlas_files = sorted(glob.glob(subdir + '/annotations_as_single_tifs/*tif'))
    test_raw_atlas_files.append(raw_atlas_files)

In [16]:
# Read in raw atlases into numpy arrays
vol_m61468_atlas = cloudvolume.CloudVolume('file:///home/ahoag/ngdemo/demo_bucket/201908_cfos/m61468_observ_rawatlas')
data_m61468_atlas = np.transpose(vol_m61468_atlas[:][...,0],(2,1,0))

Downloading: 24116it [01:43, 325.36it/s]                           


In [27]:
vol_f61465_atlas = cloudvolume.CloudVolume('file:///home/ahoag/ngdemo/demo_bucket/201908_cfos/f61465_observ_rawatlas')
data_f61465_atlas = np.transpose(vol_f61465_atlas[:][...,0],(2,1,0))

Downloading: 22758it [01:46, 156.62it/s]                           


In [17]:
data_m61468_atlas.shape

(1342, 2560, 2160)

In [19]:
# Now make sure they share the same label ids 
segments_1 = np.unique(data_m61468_atlas)

In [28]:
segments_2 = np.unique(data_f61465_atlas)

In [29]:
np.array_equal(segments_1,segments_2)

True

OK, the IDs in the two raw-space atlases are the same which is reassuring. It means that we can reliably test the align-by-region procedure described above.

Let's pick a region of interest, find its ID, then try to align based on the center of mass of that ID.

Let's choose the parabrachial nucleus (ID=867) since that is one where Kelly was looking for her signal

In [30]:
assert 867 in segments_1
assert 867 in segments_2

## Calculate centers of mass for Parabrachial nucleus (ID=867)

In [24]:
def calc_center_of_mass3D(a,label):
    """
    ---PURPOSE---
    Calculate center of mass of a label (aka "id" or "segment")
    in a 3D numpy array. 
    ---INPUT---
    a        3D numpy array
    label    An integer id that is in the array
    ---OUTPUT--
    z_avg    Average z coordinate of the label in the array
    y_avg    Average y coordinate of the label in the array
    x_avg    Average x coordinate of the label in the array
             It returns them in this order (z,y,x) to conform with output of scipy's center of mass fn
    """
    z_indices,y_indices,x_indices = np.where(a==label)
    z_avg,y_avg,x_avg = np.mean((z_indices,y_indices,x_indices),axis=1)
    return np.array([z_avg,y_avg,x_avg])

In [32]:
t1 = time.perf_counter()
center_of_mass_867_brain1 = calc_center_of_mass3D(a=data_m61468_atlas,label=867)
center_of_mass_867_brain2 = calc_center_of_mass3D(a=data_f61465_atlas,label=867)
print("#z_avg, y_avg, x_avg")
print(center_of_mass_867_brain1)
print(center_of_mass_867_brain2)
t2 = time.perf_counter()

#z_avg, y_avg, x_avg
[ 617.87333611 1824.06316462 1089.66179349]
[ 595.18092685 1810.88187478 1093.82468293]


NameError: name 'f2' is not defined

In [33]:
print(f"Took {t2-t1} seconds")

Took 37.94324218800011 seconds


Let's choose **brain 1 to be the reference brain** and shift brain 2 to align with brain 1

## Shifting brain 2 to align with brain 1

In [34]:
# First calculate the offsets
offset_zyx_float = center_of_mass_867_brain1-center_of_mass_867_brain2
offset_zyx_float

array([22.69240925, 13.18128984, -4.16288944])

In [35]:
# can only shift by integers, so will need to round these offsets
offset_zyx = [int(round(i)) for i in offset_zyx_float]
offset_zyx

[23, 13, -4]

This is how much I need to shift Brain 2 so that it is aligned with Brain 1. 

Let's test this by making a copy of atlas 2 and then shifting it by this much and then recalculating center of mass

In [55]:
t1 = time.perf_counter()
z_offset,y_offset,x_offset = offset_zyx
data_f61465_atlas_aligned2_brain1 = np.roll(data_f61465_atlas,shift=(x_offset,y_offset,z_offset),axis=(2,1,0))
t2 = time.perf_counter()
print(f"Took {t2-t1} seconds")

Took 5.439611916999638 seconds


In [37]:
# Recalculate center of mass
center_of_mass_867_brain2_shifted = calc_center_of_mass3D(a=shifted_raw_atlas_data_2,label=867)
print("#z_avg, y_avg, x_avg")
print("Brain 1:", center_of_mass_867_brain1)
print("Brain 2:",center_of_mass_867_brain2)
print("Brain 2, shifted:",center_of_mass_867_brain2_shifted)

#z_avg, y_avg, x_avg
Brain 1: [ 617.87333611 1824.06316462 1089.66179349]
Brain 2: [ 595.18092685 1810.88187478 1093.82468293]
Brain 2, shifted: [ 618.18092685 1823.88187478 1089.82468293]


OK, so it got to within a half pixel in each dimension of Brain 1.

Now let's align the actual raw data volumes

In [40]:
# First need to load in both brains
# Brain 1
vol_m61468 = cloudvolume.CloudVolume('file:///home/ahoag/ngdemo/demo_bucket/201908_cfos/190820_m61468_observ')
data_m61468 = np.transpose(vol_m61468[:][...,0],(2,1,0))

Downloading: 24172it [02:01, 166.66it/s]                           


In [39]:
# Brain 2
vol_f61465 = cloudvolume.CloudVolume('file:///home/ahoag/ngdemo/demo_bucket/201908_cfos/190821_f61465_observ/')
data_f61465 = np.transpose(vol_f61465[:][...,0],(2,1,0))

Downloading: 22766it [01:57, 165.35it/s]                           


In [42]:
data_f61465.shape

(1266, 2560, 2160)

In [43]:
data_f61465_atlas.shape

(1265, 2560, 2160)

In [44]:
data_m61468.shape

(1343, 2560, 2160)

In [45]:
data_m61468_atlas.shape

(1342, 2560, 2160)

## First lets view a single brain in Neuroglancer along with its raw data atlas 

In [66]:
# So we can compare once we hvae applied the offset 
viewer = neuroglancer.Viewer()
# This volume handle can be used to notify the viewer that the data has changed.
volume_m61468 = neuroglancer.LocalVolume(
        data=data_m61468, # need it in z,y,x order, strangely
        voxel_size=[5000,5000,5000],
        voxel_offset = [0, 0, 0], # x,y,z in nm not voxels
        volume_type='image',
         )
volume_m61468_atlas = neuroglancer.LocalVolume(
        data=data_m61468_atlas, # need it in z,y,x order, strangely
        voxel_size=[5000,5000,5000],
        voxel_offset = [0, 0, 0], # x,y,z in nm not voxels
        volume_type='segmentation',
         )
with viewer.txn() as s:
#     s.layers['m61468'] = neuroglancer.ImageLayer(
#         source=volume_m61468,
#         shader="""
#         void main() {
#         float v = toNormalized(getDataValue(0)) * 20.0;
#         emitRGBA(vec4(0.0, v, 0.0, v));
#         }
#         """,
#     )
    s.layers['atlas_m61468'] = neuroglancer.SegmentationLayer(source=volume_m61468_atlas
    )

print(viewer)

http://127.0.0.1:40244/v/e3d253a1226f664a47aafc9d400fc2489c281184/


## Now let's add the two layers for Brain 2, unshifted first

In [67]:
volume_f61465 = neuroglancer.LocalVolume(
        data=data_f61465, # need it in z,y,x order, strangely
        voxel_size=[5000,5000,5000],
        voxel_offset = [0, 0, 0], # x,y,z in nm not voxels
        volume_type='image',
         )
volume_f61465_atlas = neuroglancer.LocalVolume(
        data=data_f61465_atlas, # need it in z,y,x order, strangely
        voxel_size=[5000,5000,5000],
        voxel_offset = [0, 0, 0], # x,y,z in nm not voxels
        volume_type='segmentation',
         )
with viewer.txn() as s:
#     s.layers['f61465'] = neuroglancer.ImageLayer(
#         source=volume_f61465,
#         shader="""
#         void main() {
#         float v = toNormalized(getDataValue(0)) * 20.0;
#         emitRGBA(vec4(v, 0.0, 0.0, v));
#         }
#         """,
#     )
    s.layers['atlas_f61465'] = neuroglancer.SegmentationLayer(source=volume_f61465_atlas
    )


That really allows you to see that there is rotation or shift or both. Let's now overlay instead the shifted atlas and raw data from brain 2.
## Viewing brain 1 and brain 2 shifted to align in the parabrachial nucleus (ID=867)

In [56]:
t1 = time.perf_counter()
data_f61465_aligned2_brain1 = np.roll(data_f61465,shift=(x_offset,y_offset,z_offset),axis=(2,1,0))
t2 = time.perf_counter()
print(f"Took {t2-t1} seconds")

Took 6.800327957000263 seconds


In [62]:
volume_f61465_shifted = neuroglancer.LocalVolume(
        data=data_f61465_aligned2_brain1, # need it in z,y,x order, strangely
        voxel_size=[5000,5000,5000],
        voxel_offset = [0, 0, 0], # x,y,z in nm not voxels
        volume_type='image',
         )
volume_f61465_atlas_shifted = neuroglancer.LocalVolume(
        data=data_f61465_atlas_aligned2_brain1, # need it in z,y,x order, strangely
        voxel_size=[5000,5000,5000],
        voxel_offset = [0, 0, 0], # x,y,z in nm not voxels
        volume_type='segmentation',
         )
with viewer.txn() as s:
    s.layers['f61465'] = neuroglancer.ImageLayer(
        source=volume_f61465_shifted,
        shader="""
        void main() {
        float v = toNormalized(getDataValue(0)) * 20.0;
        emitRGBA(vec4(v, 0.0, 0.0, v));
        }
        """,
    )
    s.layers['atlas_f61465'] = neuroglancer.SegmentationLayer(source=volume_f61465_atlas_shifted
    )


In [63]:
with viewer.txn() as s:
    s.layout = neuroglancer.row_layout(
        [neuroglancer.LayerGroupViewer(layers=['f61465','atlas_f61465']),
         neuroglancer.LayerGroupViewer(layers=['m61468','atlas_m61468'])])  

Exception in callback None()
handle: <Handle cancelled>
Traceback (most recent call last):
  File "/home/ahoag/anaconda3/envs/lightserv/lib/python3.7/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "/home/ahoag/anaconda3/envs/lightserv/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 122, in _handle_events
    handler_func(fileobj, events)
  File "/home/ahoag/anaconda3/envs/lightserv/lib/python3.7/site-packages/tornado/stack_context.py", line 300, in null_wrapper
    return fn(*args, **kwargs)
  File "/home/ahoag/anaconda3/envs/lightserv/lib/python3.7/site-packages/tornado/iostream.py", line 713, in _handle_events
    self._handle_write()
  File "/home/ahoag/anaconda3/envs/lightserv/lib/python3.7/site-packages/tornado/iostream.py", line 1063, in _handle_write
    self._write_buffer.advance(num_bytes)
  File "/home/ahoag/anaconda3/envs/lightserv/lib/python3.7/site-packages/tornado/iostream.py", line 184, in advance
    assert 